# Basic Model

In [62]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,roc_curve
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import seaborn as sns
import markov
import random
from markov.api.schemas.model_recording import SingleTagInferenceRecord,RecordMetaType,RecordCustomMetric
from markov.api.recording.experiments.experiment_summary_api import ExperimentSummary

 [Elapsed Time: 0:32:29]  [|                    #               |] ( 19.5 s/B) 

### Ingesting, preprocessing, and splitting Twitter data, then converting text to TF-IDF vectors for training and validation.






In [15]:
data = pd.read_csv('../Data/Cleaned_Tweets_stopwords.csv')

X = data['cleaned_text']
y = data['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
tfidf_vectorizer = TfidfVectorizer(max_features=2500) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

In [17]:
svm_model = SVC(kernel='linear',C=1.0) 

In [18]:
import markov

my_project = markov.Project.get_by_id("vXmg4DkwjkU3JR")

recorder1 = my_project.create_experiment_recorder(
        experiment_name = "SVM(Basic)",
    experiment_notes = "Linear SVM Experiment",
    hyper_parameters ={
        "C": 1.0,
        "kernel": "linear"
    }
)
recorder1.register()

✔ Fetching project with id vXmg4DkwjkU3JR⠙ Fetching project with id vXmg4DkwjkU3JR⠹ Fetching project with id vXmg4DkwjkU3JR⠸ Fetching project with id vXmg4DkwjkU3JR⠼ Fetching project with id vXmg4DkwjkU3JR⠴ Fetching project with id vXmg4DkwjkU3JR⠦ Fetching project with id vXmg4DkwjkU3JR⠧ Fetching project with id vXmg4DkwjkU3JR⠇ Fetching project with id vXmg4DkwjkU3JR⠏ Fetching project with id vXmg4DkwjkU3JR⠋ Fetching project with id vXmg4DkwjkU3JR⠙ Fetching project with id vXmg4DkwjkU3JR⠹ Fetching project with id vXmg4DkwjkU3JR⠸ Fetching project with id vXmg4DkwjkU3JR⠼ Fetching project with id vXmg4DkwjkU3JR⠴ Fetching project with id vXmg4DkwjkU3JR⠦ Fetching project with id vXmg4DkwjkU3JR Successfully fetched project Team_2_Updated_Results
ℹ Creating MarkovML project object Team_2_Updated_Results. Call project_object.register() to register with MarkovML backend.
ℹ Creating MarkovML model object Model: SVM(Basic). Call model_object.register() to register with MarkovML backend.
✔ Creatin

In [19]:
svm_model.fit(X_train_tfidf, y_train)

SVC(kernel='linear')

In [20]:
y_pred_train = svm_model.predict(X_train_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy:", accuracy)
hinge_loss = np.maximum(0, 1 - y_train * y_pred_train).mean()
print("Hinge Loss:", hinge_loss)
training_accuracy = accuracy
training_loss = hinge_loss


Accuracy: 0.9225806451612903
Hinge Loss: 0.5525616698292221


In [21]:
y_pred = svm_model.predict(X_val_tfidf)

In [22]:
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)
hinge_loss = np.maximum(0, 1 - y_val * y_pred).mean()
print("Hinge Loss:", hinge_loss)


Accuracy: 0.84597875569044
Hinge Loss: 0.6107738998482549


In [23]:
with recorder1:
    recorder1.add_record({"accuracy": accuracy})
    recorder1.add_record({"loss": hinge_loss})
    recorder1.summary.add_training_loss(value="{:.2f}".format(training_loss))
    recorder1.summary.add_training_accuracy(value="{:.2f}".format(training_accuracy))
    recorder1.summary.add_validation_loss(value="{:.2f}".format(hinge_loss))
    recorder1.summary.add_validation_accuracy(value="{:.2f}".format(accuracy))
    



ℹ Starting recorder to track training data on markov backend using multiple threads. Please wait!


 [Elapsed Time: 0:00:00]  [|#                                   |] (  0.0 s/B) 


 [Elapsed Time: 0:00:08]  [|         #                          |] ( 12.2 B/s) 

✔ Please wait while we send the remaining data to the markov backend... Recording has successfully completed.
ℹ You can view the experiment at \https://app.markovml.com/pioneer-wsp-27a2ukk2as/proj/vXmg4DkwjkU3JR/experiments/hp-EXoyacntX2k9NC2BDxzxqzN]8;;\


### Hyperparameter tuning for SVM classification on TF-IDF text data, varying C values, n-gram ranges, and max features.

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,roc_curve
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import seaborn as sns


In [25]:
data = pd.read_csv('../Data/Cleaned_Tweets_stopwords.csv')

X = data['cleaned_text']
y = data['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
c_values = [0.01, 0.1, 1, 10, 100]
ngram_ranges = [(1, 1), (1, 2), (1, 3)]
max_features_values = [5000, 10000, 15000]

# Initialize variables to store best results
best_score = 0
best_params = {}

# Iterate over hyperparameter combinations
for c in c_values:
    for ngram_range in ngram_ranges:
        for max_features in max_features_values:

            tfidf_vectorizer = TfidfVectorizer(max_features=max_features, ngram_range=ngram_range)
            X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
            X_val_tfidf = tfidf_vectorizer.transform(X_val)
            

            # Create and train the SVM model with hyperparameter tuning
            svm_model = SVC(kernel='linear',C=c, class_weight='balanced')
            
            my_project = markov.Project.get_by_id("vXmg4DkwjkU3JR")
            recorder1 = my_project.create_experiment_recorder(
                    experiment_name = f"SVM_Parameters-{c}-{ngram_range}-{max_features}",
                experiment_notes = "Linear SVM Experiment",
                hyper_parameters ={
                    "C": c,
                    "kernel": "linear",
                    "max_features":max_features,
                    "ngram-range":ngram_range
                }
            )
            recorder1.register()
            svm_model.fit(X_train_tfidf, y_train)
            #predcting training accuracy
            y_pred_train = svm_model.predict(X_train_tfidf)
            accuracy = accuracy_score(y_train, y_pred_train)
            print("Accuracy:", accuracy)
            hinge_loss = np.maximum(0, 1 - y_train * y_pred_train).mean()
            print("Hinge Loss:", hinge_loss)
            training_accuracy = accuracy
            training_loss = hinge_loss
            
            # Predict on the validation set
            y_pred = svm_model.predict(X_val_tfidf)
            
            # Calculate accuracy
            accuracy = accuracy_score(y_val, y_pred)
            print("Accuracy:", accuracy)
            hinge_loss = np.maximum(0, 1 - y_val * y_pred).mean()
            print("Hinge Loss:", hinge_loss)

            with recorder1:
                recorder1.add_record({"accuracy": accuracy})
                recorder1.add_record({"loss": hinge_loss})
                recorder1.summary.add_training_loss(value="{:.2f}".format(training_loss))
                recorder1.summary.add_training_accuracy(value="{:.2f}".format(training_accuracy))
                recorder1.summary.add_validation_loss(value="{:.2f}".format(hinge_loss))
                recorder1.summary.add_validation_accuracy(value="{:.2f}".format(accuracy))


            # Calculate ROC AUC score
            roc_auc = roc_auc_score(y_val, y_pred)

            # Print current combination's results
            print(f'C: {c}, ngram_range: {ngram_range}, max_features: {max_features}')
            print(f'ROC AUC Score: {roc_auc}\n')
            

            # Update best results if necessary
            if roc_auc > best_score:
                best_score = roc_auc
                best_params = {'C': c, 'ngram_range': ngram_range, 'max_features': max_features}

# Print best results


⠋ Fetching project with id vXmg4DkwjkU3JR⠙ Fetching project with id vXmg4DkwjkU3JR⠹ Fetching project with id vXmg4DkwjkU3JR⠸ Fetching project with id vXmg4DkwjkU3JR⠼ Fetching project with id vXmg4DkwjkU3JR⠴ Fetching project with id vXmg4DkwjkU3JR

 [Elapsed Time: 0:01:10]  [|                #                   |] (  1.4 B/s) 

✔ Fetching project with id vXmg4DkwjkU3JR⠧ Fetching project with id vXmg4DkwjkU3JR⠇ Fetching project with id vXmg4DkwjkU3JR⠏ Fetching project with id vXmg4DkwjkU3JR⠋ Fetching project with id vXmg4DkwjkU3JR⠙ Fetching project with id vXmg4DkwjkU3JR⠹ Fetching project with id vXmg4DkwjkU3JR⠸ Fetching project with id vXmg4DkwjkU3JR⠼ Fetching project with id vXmg4DkwjkU3JR⠴ Fetching project with id vXmg4DkwjkU3JR⠦ Fetching project with id vXmg4DkwjkU3JR⠧ Fetching project with id vXmg4DkwjkU3JR⠇ Fetching project with id vXmg4DkwjkU3JR Successfully fetched project Team_2_Updated_Results
ℹ Creating MarkovML project object Team_2_Updated_Results. Call project_object.register() to register with MarkovML backend.
ℹ Creating MarkovML model object Model: SVM_Parameters-0.01-(1, 1)-5000. Call model_object.register() to register with MarkovML backend.
⠋ Creating Model Model: SVM_Parameters-0.01-(1, 1)-5000⠙ Creating Model Model: SVM_Parameters-0.01-(1, 1)-5000⠹ Creating Model Model: SVM_Parameters-0.0

 [Elapsed Time: 0:01:12]  [|   #                                |] (  1.4 B/s) 

✔ Creating Model Model: SVM_Parameters-0.01-(1, 1)-5000⠙ Creating Model Model: SVM_Parameters-0.01-(1, 1)-5000⠹ Creating Model Model: SVM_Parameters-0.01-(1, 1)-5000⠸ Creating Model Model: SVM_Parameters-0.01-(1, 1)-5000⠼ Creating Model Model: SVM_Parameters-0.01-(1, 1)-5000⠴ Creating Model Model: SVM_Parameters-0.01-(1, 1)-5000⠦ Creating Model Model: SVM_Parameters-0.01-(1, 1)-5000⠧ Creating Model Model: SVM_Parameters-0.01-(1, 1)-5000 Model creation for model Model: SVM_Parameters-0.01-(1, 1)-5000 successful!
ℹ Creating ExperimentRecorder object SVM_Parameters-0.01-(1, 1)-5000. Call recorder_object.register() to register with MarkovML backend.
⠋ Registering ExperimentRecorder with Markov Backend.⠙ Registering ExperimentRecorder with Markov Backend.⠹ Registering ExperimentRecorder with Markov Backend.⠸ Registering ExperimentRecorder with Markov Backend.⠼ Registering ExperimentRecorder with Markov Backend.⠴ Registering ExperimentRecorder with Markov Backend.⠦ Registering ExperimentReco

...

✔ Registering ExperimentRecorder with Markov Backend.⠴ Registering ExperimentRecorder with Markov Backend.⠦ Registering ExperimentRecorder with Markov Backend.⠧ Registering ExperimentRecorder with Markov Backend.⠇ Registering ExperimentRecorder with Markov Backend.⠏ Registering ExperimentRecorder with Markov Backend.⠋ Registering ExperimentRecorder with Markov Backend.⠙ Registering ExperimentRecorder with Markov Backend.⠹ Registering ExperimentRecorder with Markov Backend.⠸ Registering ExperimentRecorder with Markov Backend.⠼ Registering ExperimentRecorder with Markov Backend.⠴ Registering ExperimentRecorder with Markov Backend.⠦ Registering ExperimentRecorder with Markov Backend.⠧ Registering ExperimentRecorder with Markov Backend.⠇ Registering ExperimentRecorder with Markov Backend.⠏ Registering ExperimentRecorder with Markov Backend.⠋ Registering ExperimentRecorder with Markov Backend.⠙ Registering ExperimentRecorder with Markov Backend. ExperimentRecorder successfully registered.
ℹ

 [Elapsed Time: 0:08:48]  [|                               #    |] (  0.2 B/s) 

Accuracy: 0.9449143352851876
Hinge Loss: 0.5415311212318369
Accuracy: 0.8391502276176024
Hinge Loss: 0.6069802731411229
ℹ Starting recorder to track training data on markov backend using multiple threads. Please wait!


 [Elapsed Time: 0:00:00]  [|#                                   |] (  0.0 s/B) 
 [Elapsed Time: 0:08:58]  [|           #                        |] (  0.2 B/s) 

KeyboardInterrupt: 

In [69]:
print('Best Hyperparameters:', best_params)
print('Best ROC AUC Score:', best_score)

Best Hyperparameters: {'C': 1, 'ngram_range': (1, 1), 'max_features': 10000}
Best ROC AUC Score: 0.8497460782687238


 [Elapsed Time: 0:46:37]  [|                                #   |] ( 28.0 s/B) 

### Identifying the best hyperparameters and associated ROC AUC score and creating and testing model with it

In [70]:
data = pd.read_csv('../Data/Cleaned_Tweets_stopwords.csv')

X = data['cleaned_text']
y = data['label']

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)


 [Elapsed Time: 0:46:39]  [|            #                       |] ( 28.0 s/B) 

In [72]:
final_tfidf_vectorizer = TfidfVectorizer(max_features=best_params['max_features'], ngram_range=best_params['ngram_range'], stop_words='english')
X_train_tfidf = final_tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = final_tfidf_vectorizer.transform(X_test)
X_val_tfidf = final_tfidf_vectorizer.transform(X_val)


final_svm_model = SVC(kernel='linear',C=best_params['C'], class_weight='balanced',probability=True)
my_project = markov.Project.get_by_id("vXmg4DkwjkU3JR")
recorder1 = my_project.create_experiment_recorder(
        experiment_name = f"Best_SVM_Model",
    experiment_notes = "Linear SVM Experiment",
    hyper_parameters ={
        "C": best_params['C'],
        "kernel": "linear",
        "max_features":best_params['max_features'],
        "ngram-range":best_params['ngram_range']
    }
)
recorder1.register()

final_svm_model.fit(X_train_tfidf, y_train)
#predcting training accuracy

y_pred_train = final_svm_model.predict(X_train_tfidf)
accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy:", accuracy)
hinge_loss = np.maximum(0, 1 - y_train * y_pred_train).mean()
print("Hinge Loss:", hinge_loss)
training_accuracy = accuracy
training_loss = hinge_loss

y_pred = final_svm_model.predict(X_val_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)
hinge_loss = np.maximum(0, 1 - y_val * y_pred).mean()
print("Hinge Loss:", hinge_loss)

# Predict on the validation set


with recorder1:
    recorder1.add_record({"accuracy": accuracy})
    recorder1.add_record({"loss": hinge_loss})
    recorder1.summary.add_training_loss(value="{:.2f}".format(training_loss))
    recorder1.summary.add_training_accuracy(value="{:.2f}".format(training_accuracy))
    recorder1.summary.add_validation_loss(value="{:.2f}".format(hinge_loss))
    recorder1.summary.add_validation_accuracy(value="{:.2f}".format(accuracy))





  # Get predicted probabilities

⠋ Fetching project with id vXmg4DkwjkU3JR⠙ Fetching project with id vXmg4DkwjkU3JR⠹ Fetching project with id vXmg4DkwjkU3JR⠸ Fetching project with id vXmg4DkwjkU3JR⠼ Fetching project with id vXmg4DkwjkU3JR⠴ Fetching project with id vXmg4DkwjkU3JR⠦ Fetching project with id vXmg4DkwjkU3JR

 [Elapsed Time: 0:47:03]  [|            #                       |] ( 28.2 s/B) 

✔ Fetching project with id vXmg4DkwjkU3JR⠇ Fetching project with id vXmg4DkwjkU3JR⠏ Fetching project with id vXmg4DkwjkU3JR⠋ Fetching project with id vXmg4DkwjkU3JR⠙ Fetching project with id vXmg4DkwjkU3JR⠹ Fetching project with id vXmg4DkwjkU3JR⠸ Fetching project with id vXmg4DkwjkU3JR⠼ Fetching project with id vXmg4DkwjkU3JR⠴ Fetching project with id vXmg4DkwjkU3JR Successfully fetched project Team_2_Updated_Results
ℹ Creating MarkovML project object Team_2_Updated_Results. Call project_object.register() to register with MarkovML backend.
ℹ Creating MarkovML model object Model: Best_SVM_Model. Call model_object.register() to register with MarkovML backend.
⠋ Creating Model Model: Best_SVM_Model⠙ Creating Model Model: Best_SVM_Model⠹ Creating Model Model: Best_SVM_Model⠸ Creating Model Model: Best_SVM_Model⠼ Creating Model Model: Best_SVM_Model⠴ Creating Model Model: Best_SVM_Model⠦ Creating Model Model: Best_SVM_Model⠧ Creating Model Model: Best_SVM_Model⠇ Creating Model Model: Best_

 [Elapsed Time: 0:47:05]  [|                                #   |] ( 28.3 s/B) 

✔ Creating Model Model: Best_SVM_Model⠴ Creating Model Model: Best_SVM_Model⠦ Creating Model Model: Best_SVM_Model⠧ Creating Model Model: Best_SVM_Model⠇ Creating Model Model: Best_SVM_Model Model creation for model Model: Best_SVM_Model successful!
ℹ Creating ExperimentRecorder object Best_SVM_Model. Call recorder_object.register() to register with MarkovML backend.
⠋ Registering ExperimentRecorder with Markov Backend.⠙ Registering ExperimentRecorder with Markov Backend.⠹ Registering ExperimentRecorder with Markov Backend.⠸ Registering ExperimentRecorder with Markov Backend.⠼ Registering ExperimentRecorder with Markov Backend.⠴ Registering ExperimentRecorder with Markov Backend.⠦ Registering ExperimentRecorder with Markov Backend.⠧ Registering ExperimentRecorder with Markov Backend.⠇ Registering ExperimentRecorder with Markov Backend.⠏ Registering ExperimentRecorder with Markov Backend.⠋ Registering ExperimentRecorder with Markov Backend.⠙ Registering ExperimentRecorder with Markov Ba

...

 [Elapsed Time: 0:00:00]  [|#                                   |] (  0.0 s/B) 
 [Elapsed Time: 0:47:27]  [|                                  # |] ( 28.5 s/B) 

 [Elapsed Time: 0:00:06]  [|          #                         |] ( 16.1 B/s) 

⠙

✔Please wait while we send the remaining data to the markov backend... Recording has successfully completed.
ℹ You can view the experiment at \https://app.markovml.com/pioneer-wsp-27a2ukk2as/proj/vXmg4DkwjkU3JR/experiments/hp-3Q38KtrZ5VGPT4pLyvQ7eD5]8;;\


 [Elapsed Time: 0:47:49]  [|                              #     |] ( 28.7 s/B) 

In [73]:
y_test_pred = final_svm_model.predict(X_test_tfidf)
y_test_prob = final_svm_model.predict_proba(X_test_tfidf)
max_values = np.max(y_test_prob, axis=1)
orig_copy = [[value] for value in max_values]
y_pred = [[value] for value in y_test_pred]


 [Elapsed Time: 0:47:51]  [|          #                         |] ( 28.7 s/B) 

 [Elapsed Time: 0:48:07]  [|      #                             |] ( 28.9 s/B) 

In [74]:
from markov.api.schemas.model_recording import SingleTagInferenceRecord,RecordMetaType
from markov import EvaluationRecorder

evaluation_recorder = EvaluationRecorder(
    name=f"Evaluating Best SVM Model",
    notes=f"Testing evaluation with MarkovML",
    model_id="D9R7SvnXnvYodkiJy3",  # or my_model.model_id
    dataset_id="4Sjrh3yito3qKjujy"
    
)

evaluation_recorder.register()

ℹ Creating EvaluationRecorder object Evaluating Best SVM Model. Call recorder_object.register() to register with MarkovML backend.
⠋ Registering evaluation recorder Evaluating Best SVM Model⠙ Registering evaluation recorder Evaluating Best SVM Model⠹ Registering evaluation recorder Evaluating Best SVM Model⠸ Registering evaluation recorder Evaluating Best SVM Model⠼ Registering evaluation recorder Evaluating Best SVM Model⠴ Registering evaluation recorder Evaluating Best SVM Model⠦ Registering evaluation recorder Evaluating Best SVM Model⠧ Registering evaluation recorder Evaluating Best SVM Model⠇ Registering evaluation recorder Evaluating Best SVM Model⠏ Registering evaluation recorder Evaluating Best SVM Model⠋ Registering evaluation recorder Evaluating Best SVM Model⠙ Registering evaluation recorder Evaluating Best SVM Model⠹ Registering evaluation recorder Evaluating Best SVM Model⠸ Registering evaluation recorder Evaluating Best SVM Model⠼ Registering evaluation recorder Evaluatin

 [Elapsed Time: 0:48:09]  [|                          #         |] ( 28.9 s/B) 

✔ Registering evaluation recorder Evaluating Best SVM Model⠦ Registering evaluation recorder Evaluating Best SVM Model⠧ Registering evaluation recorder Evaluating Best SVM Model⠇ Registering evaluation recorder Evaluating Best SVM Model⠏ Registering evaluation recorder Evaluating Best SVM Model⠋ Registering evaluation recorder Evaluating Best SVM Model⠙ Registering evaluation recorder Evaluating Best SVM Model⠹ Registering evaluation recorder Evaluating Best SVM Model⠸ Registering evaluation recorder Evaluating Best SVM Model⠼ Registering evaluation recorder Evaluating Best SVM Model⠴ Registering evaluation recorder Evaluating Best SVM Model⠦ Registering evaluation recorder Evaluating Best SVM Model⠧ Registering evaluation recorder Evaluating Best SVM Model⠇ Registering evaluation recorder Evaluating Best SVM Model Evaluation recorder creation for EvaluationRecording(name='Evaluating Best SVM Model', model_id='D9R7SvnXnvYodkiJy3', note='Testing evaluation with MarkovML', dataset_id='4S

 [Elapsed Time: 0:48:17]  [|                                  # |] ( 29.0 s/B) 

In [75]:
def _get_cost(inferred, actual):
    if actual == inferred:
        return 0
    else:
        return random.randint(2, 5)
    
urid = 1
for prob, pred, orig, txt in zip(orig_copy, y_pred, y_test, X_test):
    urid=urid+1
    mi_record = SingleTagInferenceRecord(
        inferred=float(pred[0]),
        actual=float(orig),
        urid=urid,
        score=float(prob[0]),
        custom_metrics=[
            RecordCustomMetric(label="Cost", value=_get_cost(float(pred[0]), float(orig))),
            RecordCustomMetric(label="Probability", value=float(prob[0]))
        ]
    )
    evaluation_recorder.add_record(mi_record)
outcome = evaluation_recorder.finish()
print(outcome)

Upload Progress : 100%|████████████████████████| 2/2 [00:04<00:00,  2.10s/batch]


EvaluationRecordingFinishResponse(count=1977, recording_id='4N2pkoE9goR3RW4NNmmM2aG', return_code='OK', message='', run_id='uNHSdMtt7GoVnR7ptPj')


 [Elapsed Time: 0:53:02]  [|  #                                 |] ( 31.8 s/B) 

In [76]:
from markov.api.schemas.model_recording import SingleTagInferenceRecord,RecordMetaType
from markov import EvaluationRecorder

evaluation_recorder = EvaluationRecorder(
    name=f"Reddit Eval on Best SVM",
    notes=f"Testing evaluation with MarkovML",
    model_id="D9R7SvnXnvYodkiJy3",  # or my_model.model_id
    dataset_id="4Sjrh3yito3qKjujy"
    
)

evaluation_recorder.register()

ℹ Creating EvaluationRecorder object Reddit Eval on Best SVM. Call recorder_object.register() to register with MarkovML backend.
⠋ Registering evaluation recorder Reddit Eval on Best SVM⠙ Registering evaluation recorder Reddit Eval on Best SVM⠹ Registering evaluation recorder Reddit Eval on Best SVM⠸ Registering evaluation recorder Reddit Eval on Best SVM⠼ Registering evaluation recorder Reddit Eval on Best SVM

 [Elapsed Time: 0:53:04]  [|                      #             |] ( 31.8 s/B) 

⠴ Registering evaluation recorder Reddit Eval on Best SVM⠦ Registering evaluation recorder Reddit Eval on Best SVM⠧ Registering evaluation recorder Reddit Eval on Best SVM⠇ Registering evaluation recorder Reddit Eval on Best SVM⠏ Registering evaluation recorder Reddit Eval on Best SVM⠋ Registering evaluation recorder Reddit Eval on Best SVM⠙ Registering evaluation recorder Reddit Eval on Best SVM⠹ Registering evaluation recorder Reddit Eval on Best SVM⠸ Registering evaluation recorder Reddit Eval on Best SVM⠼ Registering evaluation recorder Reddit Eval on Best SVM⠴ Registering evaluation recorder Reddit Eval on Best SVM⠦ Registering evaluation recorder Reddit Eval on Best SVM⠧ Registering evaluation recorder Reddit Eval on Best SVM⠇ Registering evaluation recorder Reddit Eval on Best SVM⠏ Registering evaluation recorder Reddit Eval on Best SVM⠋ Registering evaluation recorder Reddit Eval on Best SVM⠙ Registering evaluation recorder Reddit Eval on Best SVM⠹ Registering evaluation record

 [Elapsed Time: 0:53:06]  [|                             #      |] ( 31.9 s/B) 

✔ Registering evaluation recorder Reddit Eval on Best SVM⠇ Registering evaluation recorder Reddit Eval on Best SVM⠏ Registering evaluation recorder Reddit Eval on Best SVM⠋ Registering evaluation recorder Reddit Eval on Best SVM⠙ Registering evaluation recorder Reddit Eval on Best SVM⠹ Registering evaluation recorder Reddit Eval on Best SVM⠸ Registering evaluation recorder Reddit Eval on Best SVM⠼ Registering evaluation recorder Reddit Eval on Best SVM⠴ Registering evaluation recorder Reddit Eval on Best SVM⠦ Registering evaluation recorder Reddit Eval on Best SVM⠧ Registering evaluation recorder Reddit Eval on Best SVM⠇ Registering evaluation recorder Reddit Eval on Best SVM⠏ Registering evaluation recorder Reddit Eval on Best SVM Evaluation recorder creation for EvaluationRecording(name='Reddit Eval on Best SVM', model_id='D9R7SvnXnvYodkiJy3', note='Testing evaluation with MarkovML', dataset_id='4Sjrh3yito3qKjujy', info={}) successful.


 [Elapsed Time: 0:53:38]  [|   #                                |] ( 32.2 s/B) 

In [77]:
data = pd.read_csv('../Data/Cleaned_Reddit.csv')

X_test = data['cleaned_text']
y_test = data['label']

X_test_tfidf = final_tfidf_vectorizer.transform(X_test)

y_test_pred = final_svm_model.predict(X_test_tfidf)
y_test_prob = final_svm_model.predict_proba(X_test_tfidf)
max_values = np.max(y_test_prob, axis=1)
orig_copy = [[value] for value in max_values]
y_pred = [[value] for value in y_test_pred]



 [Elapsed Time: 0:53:46]  [|            #                       |] ( 32.3 s/B) 

In [78]:
def _get_cost(inferred, actual):
    if actual == inferred:
        return 0
    else:
        return random.randint(2, 5)
    
urid = 1
for prob, pred, orig, txt in zip(orig_copy, y_pred, y_test, X_test):
    urid=urid+1
    mi_record = SingleTagInferenceRecord(
        inferred=float(pred[0]),
        actual=float(orig),
        urid=urid,
        score=float(prob[0]),
        custom_metrics=[
            RecordCustomMetric(label="Cost", value=_get_cost(float(pred[0]), float(orig))),
            RecordCustomMetric(label="Probability", value=float(prob[0]))
        ]
    )
    evaluation_recorder.add_record(mi_record)
outcome = evaluation_recorder.finish()
print(outcome)

Upload Progress : 100%|████████████████████████| 1/1 [00:01<00:00,  1.33s/batch]


EvaluationRecordingFinishResponse(count=752, recording_id='eCeNnkX8BRDwQDfYANJ4D7g', return_code='OK', message='', run_id='yJRAChfnp3m9APdWhGM')


 [Elapsed Time: 0:56:45]  [|  #                                 |] ( 34.1 s/B) 